# Extending Matplotlib
![](images/bokeh.png)


There are a number of newer libraries that have attempted to extend Python's plotting capabilities.  We will look at `bokeh`, which I think is an easy to use interface that is something similar to d3.js, but in my opinion much less verbose.

In [1]:
%%bash
pip install bokeh
pip install packaging
pip install pyyaml
pip install yaml
pip install pycosat

  Could not find a version that satisfies the requirement yaml (from versions: )
No matching distribution found for yaml


In [2]:
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
import pandas as pd

In [3]:
output_notebook()

#have to make sure call this output notebook or else it come in separate pop out

Loading BokehJS ...

In [4]:
import numpy as np

In [5]:
x = np.linspace(-3,3,100)
y = np.cos(x)

In [6]:
p = figure(width=500, height = 500)
p.circle(x, y, size = 7, color = "firebrick", alpha = 0.5)
show(p)

In [7]:
from bokeh.sampledata.autompg import autompg

In [8]:
autompg.head()

,mpg,cyl,displ,hp,weight,accel,yr,origin,name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino


In [10]:
autompg[autompg["origin"] == 3].head()

,mpg,cyl,displ,hp,weight,accel,yr,origin,name
14,24.0,4,113.0,95,2372,15.0,70,3,toyota corona mark ii
18,27.0,4,97.0,88,2130,14.5,70,3,datsun pl510
29,27.0,4,97.0,88,2130,14.5,71,3,datsun pl510
31,25.0,4,113.0,95,2228,14.0,71,3,toyota corona
52,31.0,4,71.0,65,1773,19.0,71,3,toyota corolla 1200


In [11]:
grouped = autompg.groupby('yr') #group by year
mpg = grouped['mpg'] #pull out mpg for each group, for every year info on mpg
avg = mpg.mean() #take avg of each year's mpg
std = mpg.std() #look at std of mpg's per year
years = list(grouped.groups.keys()) #use years as labels to plot with
american = autompg[autompg["origin"] == 1]
japanese = autompg[autompg["origin"] == 3]

In [12]:
p = figure(title = "MPG by Year (Japan and US)")
p.vbar(x = years, bottom = avg-std, top = avg+std, width = 0.8,
      fill_alpha = 0.2, line_color = None, legend = "MPG 1 stddev")
#creating a vert bar at each year, which tell us bottom is mean - std and the top of the bar is mean + std

p.circle(x=japanese['yr'], y=japanese['mpg'], size = 10, alpha = 0.5,
        color = 'red', legend = 'Japanese')

p.triangle(x = american['yr'], y = american['mpg'], size = 10, alpha = 0.3,
          color = 'blue', legend = 'American')

show(p)

### DataFrame Plotting

Here, we need to pass our data to the `ColumnDataSource` method, and call from here the columns of interest for a given plot.  Below, we use the `autompg` to display three different plots bsed on different variable comparisons within the data.  We can display these together using the `gridplot` argument.



In [13]:
from bokeh.models import ColumnDataSource
from bokeh.layouts import gridplot

source = ColumnDataSource(autompg)
options = dict(plot_width = 300, plot_height = 300)

#at each step instantiate a new figure object
p1 = figure(title="MPG by Year", **options )
p1.circle('yr', 'mpg', color = 'blue', source = source)

p2 = figure(title="HP vs. Displacement", **options)
p2.circle('hp', 'displ', color = 'green', source = source)

p3 = figure(title = "MPG vs. Displacement", **options)
p3.circle('mpg', 'displ', size = 'cyl', line_color = 'red', fill_color = None, source = source)

p = gridplot([[p1, p2, p3]])

show(p)

### Outputting HTML

If we want to save our plot as a standalone `.html` file, we can use the `output_file` method and direct the output to an html file of our choice.

In [14]:
from bokeh.plotting import output_file

output_file('barplot.html')
#this will output a barplot
#now you have a html file for the barplot

p = figure(title = "MPG by Year (Japan and US)")
p.vbar(x = years, bottom = avg-std, top = avg+std, width = 0.8,
      fill_alpha = 0.2, line_color = None, legend = "MPG 1 stddev")

p.circle(x=japanese['yr'], y=japanese['mpg'], size = 10, alpha = 0.5,
        color = 'red', legend = 'Japanese')

p.triangle(x = american['yr'], y = american['mpg'], size = 10, alpha = 0.3,
          color = 'blue', legend = 'American')

show(p)

### Serving Application

Finally, we can use the bokeh server to serve bokeh plots as applications.  For example, we have a weather application in the `weather` directory.  

In [13]:
%%bash
bokeh serve --show weather/

Process is terminated.
